In [1]:
from data.data_processing import Students
from mdgp import MDGP
import pandas as pd

In [5]:
# Grupos de alunos
students = Students()

# olhar turmas disponíveis
std_classes = students.get_available_classes()

# selecionar uma para usar no MDGP
std_class = std_classes[0]

students_in_class, distance_matrix = students.get_name_distance_matrix(std_class)

# dar uma olhada se da pra formar grupos
students_in_class.shape[0]


27

In [6]:
# criar instância do MDGP para resolver o problema passando a matriz de distâncias e o número de grupos
mdgp = MDGP(distance_matrix, 9)

# obter agenda de resfriamento, tem as funções alfa (1a a ser implementada) e as sugeridas nos slides: exponencial, linear e logaritmica
schedules = [
    {'cooling_function': 'alpha', 'cooling_schedule': mdgp.alpha_schedule(10000, 0.0001, 100, 0.9)},
    {'cooling_function': 'exponential', 'cooling_schedule': mdgp.exponential_schedule(10000, 0.0001, 0.5)},
    {'cooling_function': 'linear', 'cooling_schedule': mdgp.linear_schedule(10000, 0.0001, 0.9)},
    {'cooling_function': 'log', 'cooling_schedule': mdgp.log_schedule(0.0001, 0.001, 1000)}
]

# modo de comparar as funções de resfriamento
for schedule_data in schedules:
    best_cost, best = mdgp.simulated_annealing(schedule_data['cooling_schedule'])
    schedule_data['best_cost'] = best_cost
    schedule_data['best_group_division'] = best


In [7]:
# comparando os resultados
results = pd.DataFrame(schedules)
results

,cooling_function,cooling_schedule,best_cost,best_group_division
0,alpha,"[10000, 10000, 10000, 10000, 10000, 10000, 100...",707,"[0, 6, 2, 8, 1, 5, 6, 1, 7, 5, 8, 7, 3, 7, 3, ..."
1,exponential,"[10000, 5000.0, 2500.0, 1250.0, 625.0, 312.5, ...",663,"[5, 6, 8, 4, 4, 7, 4, 3, 0, 3, 7, 5, 6, 5, 0, ..."
2,linear,"[10000, 9999.1, 9998.2, 9997.3, 9996.4, 9995.5...",686,"[6, 7, 3, 4, 4, 1, 7, 5, 4, 1, 5, 0, 2, 6, 3, ..."
3,log,"[0.00014474388394765367, 0.0001447438839476536...",707,"[2, 4, 6, 7, 7, 3, 8, 2, 5, 1, 8, 4, 0, 4, 0, ..."


In [ ]:
# se quiser monitorar a convergência, fazer um plot dinâmico etc, usar o iterator do simulated annealing, por exemplo:
# usar o iterator faz ficar bem mais lento, podemos colocar pra retornar uma lista na função, mas quero deixar o iterator pra fazer plot dinâmico

for T, state_cost, state, best_cost, best in mdgp.simulated_annealing_iterator(schedules[0]['cooling_schedule']):
        print("---------------------------------")
        print(f'temperatura atual {T}')
        print(f'custo do estado atual {state_cost}')
        print(f'custo do melhor estado {best_cost}')
        print("---------------------------------")